In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

########################################################
print(tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


########################################################

text_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", text_url)

with open(filepath) as f:
    raw_text = f.read()

2.1.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6023091882978725477
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10068632535
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5739925153658600788
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Default GPU Device: /device:GPU:0


In [2]:
print(raw_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [3]:
"".join(sorted(set(raw_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([raw_text])

In [5]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [6]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [7]:
# # Creating a reverse dictionary
# reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# # Function takes a tokenized sentence and returns the words
# def sequence_to_text(list_of_indices):
#     # Looking up words in dictionary
#     words = [reverse_word_map.get(letter) for letter in list_of_indices]
#     return(words)


# sequence_to_text([50, 10, 8, 7, 3])

In [8]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

max_id, dataset_size

(39, 1)

In [27]:
# Encoding the full text (substract 1 to get IDs from 0-38 rather than 1-39)
[encoded] = np.array(tokenizer.texts_to_sequences([raw_text])) - 1
# train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

train_size = len(encoded) * 90 // 100
# dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
# [encoded] = np.array(tokenizer.texts_to_sequences([raw_text])) -1

n_steps = 100
window_length = n_steps + 1 
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
# dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset= dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:, 1:]))

dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [29]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [30]:
# model = keras.models.Sequential([
#     keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
#     keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax')),
# ])


model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     # no dropout in stateful RNN (https://github.com/ageron/handson-ml2/issues/32)
                     # dropout=0.2, recurrent_dropout=0.2,
                     ),
    keras.layers.GRU(128, return_sequences=True,
                     # dropout=0.2, recurrent_dropout=0.2
                    ),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)






# model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer="adam")


Train for 31370 steps
Epoch 1/10
31370/31370 [==============================] - 358s 11ms/step - loss: 0.9553
Epoch 2/10
31370/31370 [==============================] - 354s 11ms/step - loss: 0.9544
Epoch 3/10
31370/31370 [==============================] - 354s 11ms/step - loss: 1.0234
Epoch 4/10
31370/31370 [==============================] - 359s 11ms/step - loss: 1.0727
Epoch 5/10
31370/31370 [==============================] - 355s 11ms/step - loss: 1.1110
Epoch 6/10
31370/31370 [==============================] - 395s 13ms/step - loss: 1.1425
Epoch 7/10
31370/31370 [==============================] - 401s 13ms/step - loss: 1.1672
Epoch 8/10
31370/31370 [==============================] - 399s 13ms/step - loss: 1.1938
Epoch 9/10
31370/31370 [==============================] - 402s 13ms/step - loss: 1.2166
Epoch 10/10
31370/31370 [==============================] - 398s 13ms/step - loss: 1.2331


In [13]:
# history = model.fit(dataset, epochs = 20)

# %load_ext tensorboard
# %tensorboard --logdir=logs

In [3]:
model_name = "Stateless_RNN_2_GRU_1_TimeDist_Dense_no_dropout"

!mkdir -p model_name

# model.save(model_name) 

A subdirectory or file -p already exists.
Error occurred while processing: -p.


In [85]:
# USING THE MODEL TO PRED CHAR
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["How are y"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'o'

In [86]:
tf.random.set_seed(42)
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()


array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]],
      dtype=int64)

In [87]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [88]:
tf.random.set_seed(42)
next_char("How are yo", temperature=1)

'u'

In [89]:

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [1]:
tf.random.set_seed(42)

# print(complete_text("t", temperature=0.2))
print(complete_text("t", temperature=1))
# print(complete_text("t", temperature=2))

NameError: name 'tf' is not defined